# Simple Spread
**Objectives**: The environment has N agents and N landmarks. Agents must learn to cover all the landmarks while avoiding collisions. All agents are globally rewarded based on how far the closest agents is to each landmark (sum of the minimum distances). The agents are penalised if they collide with other agents (-1 for each collision).

**Actions**:
 - Agent observations: `[self_vel, self_pos, landmark_rel_positions, other_agent_rel_positions, communication]`
 - Agent action space: `[no_action, move_left, move_right, move_down, move_up]`

In [2]:
from pettingzoo.mpe import simple_spread_v3
num_agents = 3
env = simple_spread_v3.env(
    N=num_agents,
    max_cycles=25,
    local_ratio=0.5,
    continuous_actions=False,
    render_mode="rgb_array"
)

In [4]:
env.reset()
env.agents

['agent_0', 'agent_1', 'agent_2']

In [6]:
for agent in env.agents:
    print(agent, env.observation_space(agent))

agent_0 Box(-inf, inf, (18,), float32)
agent_1 Box(-inf, inf, (18,), float32)
agent_2 Box(-inf, inf, (18,), float32)


In [7]:
for agent in env.agents:
    print(agent, env.action_space(agent))

agent_0 Box(0.0, 1.0, (5,), float32)
agent_1 Box(0.0, 1.0, (5,), float32)
agent_2 Box(0.0, 1.0, (5,), float32)


In [13]:
actions_dict = {agent: env.action_space(agent).sample() for agent in env.agents}
actions_dict

{'agent_0': array([0.04795634, 0.5480799 , 0.96403193, 0.15368281, 0.9715407 ],
       dtype=float32),
 'agent_1': array([0.6359698 , 0.8304244 , 0.3023392 , 0.18200669, 0.73282206],
       dtype=float32),
 'agent_2': array([0.6605648 , 0.38431332, 0.78608483, 0.5651855 , 0.51340795],
       dtype=float32)}

In [16]:
obs, rew, term, trunc, info = env.step(actions_dict)
rew

defaultdict(int,
            {'agent_0': -0.9531965289987081,
             'agent_1': -0.9531965289987081,
             'agent_2': -0.9531965289987081})

In [18]:
import os
from typing import Optional, Tuple

import gymnasium as gym
import numpy as np
import torch
from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager
from tianshou.trainer import offpolicy_trainer
from tianshou.utils.net.common import Net

from pettingzoo.mpe import simple_spread_v3

In [19]:
def _get_env():
    """This functions is needed to provide callables for DummyVectorEnv"""
    return PettingZooEnv(simple_spread_v3.env(N=num_agents, max_cycles=25, local_ratio=0.5, continuous_actions=False,
                                              render_mode="rgb_array"))

In [20]:
def _get_agents(
    agent_0: Optional[BasePolicy] = None,
    agent_1: Optional[BasePolicy] = None,
    agent_2: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    env = _get_env()
    observation_space = (
        env.observation_space["observation"]
        if isinstance(env.observation_space, gym.spaces.Dict)
        else env.observation_space
    )
    if agent_0 is None:
        # model
        net = Net(
            state_shape=observation_space.shape or observation_space.n,
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=[128, 128, 128, 128],
            device="cuda" if torch.cuda.is_available() else "cpu"
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=1e-4)
        agent_0 = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor=0.9,
            estimation_step=3,
            target_update_freq=320
        )

    if agent_1 is None:
        # model
        net = Net(
            state_shape=observation_space.shape or observation_space.n,
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=[128, 128, 128, 128],
            device="cuda" if torch.cuda.is_available() else "cpu"
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=1e-4)
        agent_1 = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor=0.9,
            estimation_step=3,
            target_update_freq=320
        )

    if agent_2 is None:
        # model
        net = Net(
            state_shape=observation_space.shape or observation_space.n,
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=[128, 128, 128, 128],
            device="cuda" if torch.cuda.is_available() else "cpu"
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=1e-4)
        agent_2 = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor=0.9,
            estimation_step=3,
            target_update_freq=320
        )
    
    agents = [agent_0, agent_1, agent_2]
    policy = MultiAgentPolicyManager(agents, env)
    return policy, optim, env.agents

In [21]:
# step 1: environment setup
train_envs = DummyVectorEnv([_get_env for _ in range(10)])
test_envs = DummyVectorEnv([_get_env for _ in range(10)])

# seed
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)
train_envs.seed(seed)
test_envs.seed(seed)

# step 2: agent setup
policy, optim, agents = _get_agents()

# step 3: collector setup
train_collector = Collector(
    policy,
    train_envs,
    VectorReplayBuffer(20_000, len(train_envs)),
    exploration_noise=True
)
test_collector = Collector(policy, test_envs, exploration_noise=True)
# policy.set_eps(1)
train_collector.collect(n_step=64 * 10)  # batch size * training_num

{'n/ep': 0,
 'n/st': 640,
 'rews': array([], dtype=float64),
 'lens': array([], dtype=int64),
 'idxs': array([], dtype=int64),
 'rew': 0,
 'len': 0,
 'rew_std': 0,
 'len_std': 0}

In [22]:
# step 4: callback functions setup
def save_best_fn(policy):
    model_save_path = os.path.join("log", "ttt", "dqn", "policy.pth")
    os.makedirs(os.path.join("log", "ttt", "dqn"), exist_ok=True)
    torch.save(policy.policies[agents[1]].state_dict(), model_save_path)

def stop_fn(mean_rewards):
    return mean_rewards >= 0.6

def train_fn(epoch, env_step):
    policy.policies[agents[1]].set_eps(0.1)

def test_fn(epoch, env_step):
    policy.policies[agents[1]].set_eps(0.05)

def reward_metric(rews):
    return rews[:, 1]

In [23]:
# step 5: run the trainer
result = offpolicy_trainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=50,
    step_per_epoch=1000,
    step_per_collect=50,
    episode_per_test=10,
    batch_size=64,
    train_fn=train_fn,
    test_fn=test_fn,
    stop_fn=stop_fn,
    save_best_fn=save_best_fn,
    update_per_step=0.1,
    test_in_train=False,
    reward_metric=reward_metric
)

Epoch #1: 1001it [00:02, 426.11it/s, agent_0/loss=2.717, agent_1/loss=3.971, agent_2/loss=4.893, env_step=1000, len=75, n/ep=0, n/st=50, rew=-42.51]                          


Epoch #1: test_reward: -54.758826 ± 6.949453, best_reward: -54.758826 ± 6.949453 in #1


Epoch #2: 1001it [00:01, 761.58it/s, agent_0/loss=0.384, agent_1/loss=3.748, agent_2/loss=4.782, env_step=2000, len=75, n/ep=0, n/st=50, rew=-50.09]                          


Epoch #2: test_reward: -49.356117 ± 15.470015, best_reward: -49.356117 ± 15.470015 in #2


Epoch #3: 1001it [00:01, 817.88it/s, agent_0/loss=0.175, agent_1/loss=4.299, agent_2/loss=5.307, env_step=3000, len=75, n/ep=0, n/st=50, rew=-56.99]                          


...

Epoch #48: test_reward: -25.677173 ± 4.513942, best_reward: -22.464294 ± 5.597708 in #35


Epoch #49: 1001it [00:01, 791.36it/s, agent_0/loss=0.046, agent_1/loss=0.973, agent_2/loss=1.237, env_step=49000, len=75, n/ep=0, n/st=50, rew=-25.48]                           


Epoch #49: test_reward: -26.104573 ± 7.729232, best_reward: -22.464294 ± 5.597708 in #35


Epoch #50: 1001it [00:01, 791.19it/s, agent_0/loss=0.037, agent_1/loss=0.976, agent_2/loss=1.190, env_step=50000, len=75, n/ep=0, n/st=50, rew=-23.07]                          


Epoch #50: test_reward: -27.509545 ± 4.035955, best_reward: -22.464294 ± 5.597708 in #35


In [24]:
# return result, policy.policies[agents[1]]
print(f"\n==========Result==========\n{result}")


==========Result==========
{'duration': '81.47s', 'train_time/model': '52.40s', 'test_step': 38250, 'test_episode': 510, 'test_time': '11.27s', 'test_speed': '3393.63 step/s', 'best_reward': -22.464293915369872, 'best_result': '-22.46 ± 5.60', 'train_step': 50000, 'train_episode': 670, 'train_time/collector': '17.80s', 'train_speed': '712.30 step/s'}
